In [3]:
import os.path as osp

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import SplineConv
from torch_geometric.typing import WITH_TORCH_SPLINE_CONV

In [4]:
if not WITH_TORCH_SPLINE_CONV:
    quit("This example requires 'torch-spline-conv'")

In [5]:
dataset = 'Cora'
transform = T.Compose([
    T.RandomNodeSplit(num_val=2000, num_test=500),
    T.TargetIndegree(),
])
path = ('./dataset/cora')
dataset = Planetoid(path, dataset, transform=transform)
data = dataset[0]
print("Number of graphs in the dataset:", data)

Number of graphs in the dataset: Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_attr=[10556, 1])


In [11]:
data.x

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [12]:
data.edge_index

tensor([[ 633, 1862, 2582,  ...,  598, 1473, 2706],
        [   0,    0,    0,  ..., 2707, 2707, 2707]])

In [15]:
data.y[100:200]

tensor([0, 2, 1, 1, 1, 2, 6, 5, 6, 1, 2, 2, 1, 5, 5, 5, 6, 5, 6, 5, 5, 1, 6, 6,
        1, 5, 1, 6, 5, 5, 5, 1, 5, 1, 1, 1, 1, 1, 1, 1, 4, 3, 0, 3, 6, 6, 0, 3,
        4, 0, 3, 4, 4, 1, 2, 2, 2, 3, 3, 3, 3, 0, 4, 5, 0, 3, 4, 3, 3, 3, 2, 3,
        3, 2, 2, 6, 1, 4, 3, 3, 3, 6, 3, 3, 3, 3, 0, 4, 2, 2, 6, 5, 3, 5, 4, 0,
        4, 3, 4, 4])

In [17]:
data.edge_attr

tensor([[0.0179],
        [0.0179],
        [0.0179],
        ...,
        [0.0238],
        [0.0238],
        [0.0238]])

In [12]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SplineConv(dataset.num_features, 16, dim=1, kernel_size=2, is_open_spline=False, degree=1, aggr="mean", root_weight=True, bias=False)
        self.conv2 = SplineConv(16, dataset.num_classes, dim=1, kernel_size=2, is_open_spline=False, degree=1, aggr="mean", root_weight=True, bias=False)

    def forward(self):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = F.dropout(x, training=self.training)
        x = F.elu(self.conv1(x, edge_index, edge_attr))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_attr)
        return F.log_softmax(x, dim=1)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)

In [14]:
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    log_probs, accs = model(), []
    for _, mask in data('train_mask', 'test_mask'):
        pred = log_probs[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [15]:

for epoch in range(1, 1001):
    train()
    train_acc, test_acc = test()
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Train: 0.3750, Test: 0.3300
Epoch: 002, Train: 0.3654, Test: 0.3160
Epoch: 003, Train: 0.3942, Test: 0.3220
Epoch: 004, Train: 0.4183, Test: 0.3300
Epoch: 005, Train: 0.4519, Test: 0.3400
Epoch: 006, Train: 0.4904, Test: 0.3600
Epoch: 007, Train: 0.5288, Test: 0.3760
Epoch: 008, Train: 0.5817, Test: 0.3980
Epoch: 009, Train: 0.6250, Test: 0.4380
Epoch: 010, Train: 0.6875, Test: 0.5040
Epoch: 011, Train: 0.7308, Test: 0.5700
Epoch: 012, Train: 0.7692, Test: 0.6420
Epoch: 013, Train: 0.8029, Test: 0.6820
Epoch: 014, Train: 0.8029, Test: 0.7160
Epoch: 015, Train: 0.8269, Test: 0.7340
Epoch: 016, Train: 0.8606, Test: 0.7360
Epoch: 017, Train: 0.8846, Test: 0.7420
Epoch: 018, Train: 0.8942, Test: 0.7560
Epoch: 019, Train: 0.9087, Test: 0.7700
Epoch: 020, Train: 0.9183, Test: 0.7820
Epoch: 021, Train: 0.9183, Test: 0.7820
Epoch: 022, Train: 0.9327, Test: 0.7880
Epoch: 023, Train: 0.9375, Test: 0.7960
Epoch: 024, Train: 0.9423, Test: 0.7960
Epoch: 025, Train: 0.9423, Test: 0.7980


In [19]:
import torch
from torch_geometric.nn import SplineConv
import torch_geometric.data as Data

# Example node features and graph connectivity
x = torch.randn((10, 16))  # 10 nodes, 16 features each
edge_index = torch.tensor([[0, 1, 2, 3], [1, 2, 3, 0]], dtype=torch.long)  # 4 edges in the graph

# Pseudo-coordinates for each edge
edge_attr = torch.rand((4, 2))  # 4 edges, each with 2-dimensional pseudo-coordinates

# Define SplineConv layer
conv = SplineConv(in_channels=16, out_channels=32, dim=2, kernel_size=5)

# Apply the convolution
out = conv(x, edge_index, edge_attr)

print(out)  # Expected output shape: [10, 32] (10 nodes, 32 output features)

tensor([[ 6.7955e-01,  5.2650e-01,  1.1339e-01, -4.5549e-01, -4.7709e-01,
          1.1157e+00, -2.0037e-01, -1.0702e-01, -2.6171e-01, -1.2348e+00,
         -6.5743e-01,  1.0021e+00,  9.7041e-01, -2.9932e-01, -1.3928e+00,
         -7.9333e-01,  7.9534e-02,  5.6862e-01, -4.4389e-01, -1.8836e-01,
         -2.0060e-01,  3.5174e-01,  6.6780e-01,  1.2698e+00,  5.9740e-01,
         -7.7514e-01,  9.6933e-01,  5.6613e-02,  1.3931e-01, -4.8644e-01,
          1.4195e-01,  4.6210e-01],
        [-2.7819e-01, -3.9711e-01, -8.7735e-02, -1.3589e+00,  4.2768e-01,
          9.7249e-02,  3.0403e-01, -6.7394e-01,  1.6640e-01,  4.5020e-01,
         -3.2768e-01, -4.0983e-01,  2.4673e-01, -4.1974e-01,  3.7472e-01,
         -4.1944e-01,  7.3404e-01, -1.1718e+00, -4.8982e-01,  2.9222e-01,
          3.2657e-01, -9.6387e-02, -2.0908e-01, -1.0209e+00,  1.1339e-01,
          2.0566e-01,  1.0735e-01, -2.6636e-01,  4.9036e-01, -1.0287e+00,
         -4.3026e-01,  7.5279e-01],
        [-2.7973e-02, -4.0241e-01, -7.06